In [1]:
import numpy as np
from scipy.stats import lognorm
import networkx as nx
import geopandas as gpd
import matplotlib.pylab as plt
import warnings
import wntr
import folium
from pyproj import Transformer
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import branca.colormap as bcm
# Suppress warning messages that will be addressed in future WNTR releases
warnings.filterwarnings("ignore", message="Column names longer than 10 characters will be truncated when saved to "
            "ESRI Shapefile.")
warnings.filterwarnings("ignore", message="'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.")
warnings.filterwarnings("ignore", message="Normalized/laundered field name:")
warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.")

# Simulação Original

In [2]:
# Create a WaterNetworkModel from an EPANET INP file

wn = wntr.network.WaterNetworkModel('PK_QD28_TESTE_DOS_NOS_DUPLICADOS (20).inp')
nome_reservatorio = 'R1'         # Nome do reservatório
base_head = 1159.99              # Carga hidráulica (ajuste conforme sua rede)
coordinates = (181765.717, 8236835.008)             # Coordenadas (opcional, ajuste conforme necessário)
# Parâmetros da tubulação
nome_tubulacao = 'P_R1_N49'
comprimento = 0.10                # Comprimento em metros (ajuste conforme necessário)
diametro = 110                   # Diâmetro em mm (ajuste conforme necessário)
rugosidade = 140                 # Rugosidade (ajuste conforme necessário)

# Adiciona o reservatório à rede
wn.add_reservoir(nome_reservatorio, base_head=base_head, coordinates=coordinates)
# Adiciona a tubulação conectando o reservatório ao nó N49
wn.add_pipe(nome_tubulacao, nome_reservatorio, 'N49', length=comprimento, diameter=diametro, roughness=rugosidade)

sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# Extraia pressões do último instante simulado
pressure = results.node['pressure'].iloc[-1]
demanda = results.node['demand'].iloc[-1]

# Converta as Coordenadas dos Nós para Latitude/Longitude
from pyproj import Transformer

transformer = Transformer.from_crs('EPSG:31983', 'EPSG:4326', always_xy=True)
nodes_latlon = {}
for node in wn.node_name_list:
    x, y = wn.get_node(node).coordinates
    lon, lat = transformer.transform(x, y)
    nodes_latlon[node] = (lat, lon)

# Normaliza as pressões para escala de cores
press_values = list(pressure.values)
# norm = colors.Normalize(vmin=min(press_values), vmax=max(press_values))
# Supondo que press_values já é sua lista de pressões
vmin = min(press_values)
vmax = max(press_values)
# Crie o mapa centralizado em um ponto da rede
lat_centro, lon_centro = list(nodes_latlon.values())[0]
m = folium.Map(location=[lat_centro, lon_centro], zoom_start=15)
velocity = results.link['velocity']

# Adicionar os trechos de redes
for pipe_name in wn.pipe_name_list:
    pipe = wn.get_link(pipe_name)
    start_node = pipe.start_node_name
    end_node = pipe.end_node_name
    latlon_start = nodes_latlon[start_node]
    latlon_end = nodes_latlon[end_node]
    velocity_last = velocity[pipe_name].iloc[-1]
    popup_text = f"{pipe_name}: Velocidade = {velocity_last:.2f} m/s"
    folium.PolyLine(
        locations=[latlon_start, latlon_end],
        color='black',
        weight=3,
        opacity=0.7,
        popup=popup_text
    ).add_to(m)


# Adicione cada nó com cor conforme a pressão

vmin = min(press_values)
vmax = max(press_values)
cmap = plt.colormaps['YlOrBr']
colors = [cmap(i / 10) for i in range(11)]
hex_colors = [mcolors.rgb2hex(c) for c in colors]

colormap = bcm.LinearColormap(hex_colors, vmin=vmin, vmax=vmax, caption='Pressão (m)')
colormap.add_to(m)

for node, (lat, lon) in nodes_latlon.items():
    press = pressure[node]
    deman = demanda[node]
    color = colormap(press)
    if node == nome_reservatorio:
        folium.Marker(
            location=[lat, lon],
            popup=f'Reservatório {node}: {press:.2f} m - Demanda {deman:.5f} l/s',
            icon=folium.Icon(color='blue', icon='tint', prefix='fa')
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=f'{node}: {press:.2f} m - Demanda {deman:.5f} l/s'
        ).add_to(m)
m.save('index.html')
m.save('rede_pressao_mapa.html')


# Gere o mapa interativo e salve em um arquivo HTML
# wntr.graphics.plot_leaflet_network(wn, filename='rede_mapa.html')



# Simulação Com Vazamento

In [3]:
import wntr
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import branca.colormap as bcm
from pyproj import Transformer

# Carregue o modelo da rede
wn = wntr.network.WaterNetworkModel('PK_QD28_TESTE_DOS_NOS_DUPLICADOS (20).inp')

# Parâmetros do reservatório (ajuste conforme necessário)
nome_reservatorio = 'R1'
base_head = 1149.99
coordinates = (181765.717, 8236835.008)

# Adiciona o reservatório à rede
wn.add_reservoir(nome_reservatorio, base_head=base_head, coordinates=coordinates)

# Adiciona a tubulação conectando o reservatório ao nó N49 (ajuste conforme necessário)
nome_tubulacao = 'P_R1_N49'
comprimento = 0.10
diametro = 0.11
rugosidade = 140
wn.add_pipe(nome_tubulacao, nome_reservatorio, 'N49', length=comprimento, diameter=diametro, roughness=rugosidade)

# --- Simulação de vazamento no trecho P39 ---

# Parâmetros do vazamento
nome_trecho = 'P17'
nome_no_inicio = wn.get_link(nome_trecho).start_node_name
nome_no_fim = wn.get_link(nome_trecho).end_node_name
comprimento_total = wn.get_link(nome_trecho).length
diametro_trecho = wn.get_link(nome_trecho).diameter
rugosidade_trecho = wn.get_link(nome_trecho).roughness

# Remove o trecho original
wn.remove_link(nome_trecho)

# Crie um nó intermediário para o vazamento
nome_vazamento = 'Leak_P17'
# Coordenadas intermediárias (ajuste conforme necessário)
x_inicio, y_inicio = wn.get_node(nome_no_inicio).coordinates
x_fim, y_fim = wn.get_node(nome_no_fim).coordinates
x_vaz, y_vaz = (x_inicio + x_fim) / 2, (y_inicio + y_fim) / 2

# Adiciona o nó do vazamento com demanda elevada (simulando o rompimento)
demanda_vazamento = 0.1  # m³/h ou L/s, ajuste conforme escala da sua rede
wn.add_junction(nome_vazamento, base_demand=demanda_vazamento, demand_pattern=None, coordinates=(x_vaz, y_vaz))

# Divide o trecho original em dois
wn.add_pipe('P17_1', nome_no_inicio, nome_vazamento, length=comprimento_total/2, diameter=diametro_trecho, roughness=rugosidade_trecho)
wn.add_pipe('P17_2', nome_vazamento, nome_no_fim, length=comprimento_total/2, diameter=diametro_trecho, roughness=rugosidade_trecho)

#Mudando o diametro de toda a rede
# Supondo que 'wn' é seu WaterNetworkModel já carregado
# novo_diametro_mm = 60  # exemplo: 150 mm
# novo_diametro_m = novo_diametro_mm / 1000.0

# for pipe_name in wn.pipe_name_list:
#     pipe = wn.get_link(pipe_name)
#     pipe.diameter = novo_diametro_m


# --- Simulação hidráulica ---
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# Extraia pressões do último instante simulado
pressure = results.node['pressure'].iloc[-1]
demanda = results.node['demand'].iloc[-1]
# Converta as coordenadas dos nós para Latitude/Longitude
transformer = Transformer.from_crs('EPSG:31983', 'EPSG:4326', always_xy=True)
nodes_latlon = {}
for node in wn.node_name_list:
    x, y = wn.get_node(node).coordinates
    lon, lat = transformer.transform(x, y)
    nodes_latlon[node] = (lat, lon)

# Normaliza as pressões para escala de cores
press_values = list(pressure.values)
vmin = min(press_values)
vmax = max(press_values)
cmap = plt.colormaps['inferno']
colors = [cmap(i / 10) for i in range(11)]
hex_colors = [mcolors.rgb2hex(c) for c in colors]
colormap = bcm.LinearColormap(hex_colors, vmin=vmin, vmax=vmax, caption='Pressão (m)')

# Crie o mapa centralizado em um ponto da rede
lat_centro, lon_centro = list(nodes_latlon.values())[0]
m = folium.Map(location=[lat_centro, lon_centro], zoom_start=15)
colormap.add_to(m)

velocity = results.link['velocity']

# Adicionar os trechos de redes
for pipe_name in wn.pipe_name_list:
    pipe = wn.get_link(pipe_name)
    start_node = pipe.start_node_name
    end_node = pipe.end_node_name
    latlon_start = nodes_latlon[start_node]
    latlon_end = nodes_latlon[end_node]
    velocity_last = velocity[pipe_name].iloc[-1]
    popup_text = f"{pipe_name}: Velocidade = {velocity_last:.2f} m/s"
    folium.PolyLine(
        locations=[latlon_start, latlon_end],
        color='black',
        weight=3,
        opacity=0.7,
        popup=popup_text
    ).add_to(m)

# Adicione cada nó com cor conforme a pressão
for node, (lat, lon) in nodes_latlon.items():
    press = pressure[node]
    deman = demanda[node]
    color = colormap(press)
    if node == nome_reservatorio:
        folium.Marker(
            location=[lat, lon],
            popup=f'Reservatório {node}: {press:.2f} m - Demanda {deman:.2f} l/s',
            icon=folium.Icon(color='blue', icon='tint', prefix='fa')
        ).add_to(m)
    elif node == nome_vazamento:
        folium.Marker(
            location=[lat, lon],
            popup=f'Vazamento {node}: {press:.2f} m - Demanda {deman:.2f} l/s',
            icon=folium.Icon(color='red', icon='exclamation-triangle', prefix='fa')
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=f'{node}: {press:.2f} m  - Demanda {deman:.2f} l/s'
        ).add_to(m)

# Salve o mapa interativo
m.save('rede_pressao_vazamento.html')


In [4]:
results.link['velocity']

name,P8,P9,P10,P11,P12,P13,P14,P15,P16,P18,...,P538,P539,P540,P541,P542,P543,P544,P_R1_N49,P17_1,P17_2
0,10.713434,10.713434,10.713434,10.713434,10.705353,10.705353,10.705353,10.705353,10.697865,0.17066,...,0.00319,0.008259,0.019709,0.031159,0.042609,0.054059,10.723228,11.074131,10.693244,0.17066


In [5]:
for pipe_name in wn.pipe_name_list:
    print(pipe_name, wn.get_link(pipe_name).velocity)


P8 None
P9 None
P10 None
P11 None
P12 None
P13 None
P14 None
P15 None
P16 None
P18 None
P19 None
P20 None
P21 None
P22 None
P23 None
P24 None
P25 None
P26 None
P27 None
P28 None
P29 None
P30 None
P31 None
P32 None
P33 None
P34 None
P35 None
P36 None
P37 None
P38 None
P39 None
P40 None
P41 None
P42 None
P43 None
P44 None
P45 None
P48 None
P49 None
P50 None
P51 None
P52 None
P53 None
P54 None
P55 None
P56 None
P57 None
P58 None
P59 None
P60 None
P61 None
P62 None
P63 None
P64 None
P65 None
P66 None
P67 None
P68 None
P69 None
P70 None
P71 None
P72 None
P73 None
P74 None
P75 None
P76 None
P77 None
P78 None
P79 None
P80 None
P81 None
P82 None
P83 None
P84 None
P85 None
P86 None
P87 None
P88 None
P89 None
P90 None
P91 None
P92 None
P93 None
P94 None
P95 None
P96 None
P97 None
P98 None
P99 None
P100 None
P101 None
P102 None
P103 None
P104 None
P105 None
P106 None
P107 None
P108 None
P109 None
P110 None
P111 None
P112 None
P113 None
P114 None
P115 None
P116 None
P117 None
P118 None
P119 None
P

# simulação com vrp

In [6]:
pipe = wn.get_link('P378')
start_node = pipe.start_node_name
end_node = pipe.end_node_name


In [7]:
pipe

<Pipe 'P378' from 'N353' to 'N354', length=45.869007359, diameter=0.032, roughness=140.0, minor_loss=0.0, check_valve=False, status=Open>

In [8]:
from math import sqrt

# Substitua pelos valores reais das coordenadas (x, y) dos nós
coords = {
    'N17': (182325.334803334, 8236241.429658412),
    'N354': (181617.609217913, 8236027.916901818),
    'N351': (181673.985346249, 8236280.518389829)
}

def distancia(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

comprimento_1 = distancia(coords['N17'], coords['N354'])    # ≈ 15,13 m
comprimento_2 = distancia(coords['N354'], coords['N351'])   # ≈ 22,36 m
fator_escala = 188.12067746 / (comprimento_1 + comprimento_2)
comprimento_1_ajustado = comprimento_1 * fator_escala
comprimento_2_ajustado = comprimento_2 * fator_escala


In [9]:
# Create a WaterNetworkModel from an EPANET INP file
import numpy as np
from scipy.stats import lognorm
import networkx as nx
import geopandas as gpd
import matplotlib.pylab as plt
import warnings
import wntr
import folium
from pyproj import Transformer
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import branca.colormap as bcm
# Suppress warning messages that will be addressed in future WNTR releases
warnings.filterwarnings("ignore", message="Column names longer than 10 characters will be truncated when saved to "
            "ESRI Shapefile.")
warnings.filterwarnings("ignore", message="'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.")
warnings.filterwarnings("ignore", message="Normalized/laundered field name:")
warnings.filterwarnings("ignore", message="Geometry is in a geographic CRS.")
wn = wntr.network.WaterNetworkModel('PK_QD28_TESTE_DOS_NOS_DUPLICADOS (20).inp')
nome_reservatorio = 'R1'         # Nome do reservatório
base_head = 1156.99              # Carga hidráulica (ajuste conforme sua rede)
coordinates = (181765.717, 8236835.008)             # Coordenadas (opcional, ajuste conforme necessário)
# Parâmetros da tubulação
nome_tubulacao = 'P_R1_N49'
comprimento = 0.10                # Comprimento em metros (ajuste conforme necessário)
diametro = 110                   # Diâmetro em mm (ajuste conforme necessário)
rugosidade = 140                 # Rugosidade (ajuste conforme necessário)

# Adiciona o reservatório à rede
wn.add_reservoir(nome_reservatorio, base_head=base_head, coordinates=coordinates)
# Adiciona a tubulação conectando o reservatório ao nó N49
wn.add_pipe(nome_tubulacao, nome_reservatorio, 'N49', length=comprimento, diameter=diametro, roughness=rugosidade)
pipe = wn.get_link('P379')
start_node = pipe.start_node_name
end_node = pipe.end_node_name
wn.remove_link('P379')

wn.add_valve(
    name='VRP_P379',
    start_node_name=start_node,
    end_node_name=end_node,
    diameter=pipe.diameter,
    valve_type='PRV',           # Tipo de válvula: PRV para redutora de pressão
    minor_loss=10.0,             # Ajuste conforme necessário
    initial_setting=20.0,       # Pressão de saída desejada em metros (exemplo)
    initial_status='Active'       # Opened, Closed, Active
)
# Parâmetros
rugosidade = 140
diametro_mm = 32
diametro_m = diametro_mm / 1000.0

# Remova o trecho antigo
wn.remove_link('P375')

# Adicione os dois novos trechos
wn.add_pipe('P375_1', 'N17', 'N354', length=comprimento_1_ajustado, diameter=diametro_m, roughness=rugosidade)
wn.add_pipe('P375_2', 'N354', 'N351', length=comprimento_2_ajustado, diameter=diametro_m, roughness=rugosidade)

pipe = wn.get_link('P375_1')
start_node = pipe.start_node_name
end_node = pipe.end_node_name
wn.remove_link('P375_1')

wn.add_valve(
    name='VRP_P375_1',
    start_node_name=start_node,
    end_node_name=end_node,
    diameter=pipe.diameter,
    valve_type='PRV',           # Tipo de válvula: PRV para redutora de pressão
    minor_loss=10.0,             # Ajuste conforme necessário
    initial_setting=20.0,       # Pressão de saída desejada em metros (exemplo)
    initial_status='Active'       # Opened, Closed, Active
)

pipe = wn.get_link('P366')
start_node = pipe.start_node_name
end_node = pipe.end_node_name
wn.remove_link('P366')

wn.add_valve(
    name='VRP_P366',
    start_node_name=start_node,
    end_node_name=end_node,
    diameter=pipe.diameter,
    valve_type='PRV',           # Tipo de válvula: PRV para redutora de pressão
    minor_loss=10.0,             # Ajuste conforme necessário
    initial_setting=20.0,       # Pressão de saída desejada em metros (exemplo)
    initial_status='Active'       # Opened, Closed, Active
)


sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

# Extraia pressões do último instante simulado
pressure = results.node['pressure'].iloc[-1]
demanda = results.node['demand'].iloc[-1]
flowrate = results.link['flowrate']
flow_last = flowrate.iloc[-1]
# Substitui valores negativos por zero
pressure = pressure.clip(lower=0)

# Converta as Coordenadas dos Nós para Latitude/Longitude
from pyproj import Transformer

transformer = Transformer.from_crs('EPSG:31983', 'EPSG:4326', always_xy=True)
nodes_latlon = {}
for node in wn.node_name_list:
    x, y = wn.get_node(node).coordinates
    lon, lat = transformer.transform(x, y)
    nodes_latlon[node] = (lat, lon)

# Normaliza as pressões para escala de cores
press_values = list(pressure.values)
# norm = colors.Normalize(vmin=min(press_values), vmax=max(press_values))
# Supondo que press_values já é sua lista de pressões
vmin = min(press_values)
vmax = max(press_values)
# Crie o mapa centralizado em um ponto da rede
lat_centro, lon_centro = list(nodes_latlon.values())[0]
m = folium.Map(location=[lat_centro, lon_centro], zoom_start=15)
velocity = results.link['velocity']


for valve_name in wn.valve_name_list:
    valve = wn.get_link(valve_name)
    start_node = valve.start_node_name
    end_node = valve.end_node_name
    latlon_start = nodes_latlon[start_node]
    latlon_end = nodes_latlon[end_node]
    folium.PolyLine(
        locations=[latlon_start, latlon_end],
        color='black',
        weight=3,
        opacity=0.8,
        popup=f'Válvula {valve_name} ({valve.valve_type})'
    ).add_to(m)

# Adicionar os trechos de redes
for pipe_name in wn.pipe_name_list:
    pipe = wn.get_link(pipe_name)
    start_node = pipe.start_node_name
    end_node = pipe.end_node_name
    latlon_start = nodes_latlon[start_node]
    latlon_end = nodes_latlon[end_node]
    velocity_last = velocity[pipe_name].iloc[-1]*3.6
    flow_value = flow_last[pipe_name]*1000
    popup_text = f"{pipe_name}: Velocidade = {velocity_last:.2f} m/s - Vazão = {flow_value:.2f} l/s"

    folium.PolyLine(
        locations=[latlon_start, latlon_end],
        color='black',
        weight=3,
        opacity=0.7,
        popup=popup_text
    ).add_to(m)


# Adicione cada nó com cor conforme a pressão

vmin = min(press_values)
vmax = max(press_values)
cmap = plt.colormaps['YlOrBr']
colors = [cmap(i / 10) for i in range(11)]
hex_colors = [mcolors.rgb2hex(c) for c in colors]

colormap = bcm.LinearColormap(hex_colors, vmin=vmin, vmax=vmax, caption='Pressão (m)')
colormap.add_to(m)

# Adiciona um ícone no ponto médio de cada VRP
for valve_name in wn.valve_name_list:
    valve = wn.get_link(valve_name)
    start_node = valve.start_node_name
    end_node = valve.end_node_name
    latlon_start = nodes_latlon[start_node]
    latlon_end = nodes_latlon[end_node]
    # Calcula o ponto médio para posicionar o ícone
    lat_valve = (latlon_start[0] + latlon_end[0]) / 2
    lon_valve = (latlon_start[1] + latlon_end[1]) / 2
    folium.Marker(
        location=[lat_valve, lon_valve],
        popup=f'VRP: {valve_name} ({valve.valve_type})',
        icon=folium.CustomIcon('https://raw.githubusercontent.com/kaioribeiro97/WNTR/f0c8942f2398fb38053519aac3e8560e4f609220/imagens/1.svg',
    icon_size=(40, 40),icon_anchor=(20, 40),
)
    ).add_to(m)


for node, (lat, lon) in nodes_latlon.items():
    press = pressure[node]
    deman = demanda[node]
    color = colormap(press)
    if node == nome_reservatorio:
        folium.Marker(
            location=[lat, lon],
            popup=f'Reservatório {node}: {press:.2f} m - Demanda {deman:.5f} l/s',
            icon=folium.CustomIcon('https://raw.githubusercontent.com/kaioribeiro97/WNTR/f0c8942f2398fb38053519aac3e8560e4f609220/imagens/2.svg',
    icon_size=(40, 40),icon_anchor=(20, 40),)
        ).add_to(m)
    else:
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=f'{node}: {press:.2f} m - Demanda {deman:.5f} l/s'
        ).add_to(m)

m.save('rede_pressao_mapa_VRP.html')


# Gere o mapa interativo e salve em um arquivo HTML
# wntr.graphics.plot_leaflet_network(wn, filename='rede_mapa.html')



In [16]:
# Listar pressões ordenadas
for node, press in sorted(pressure.items(), key=lambda x: x[1], reverse=True):
    print(f'Nó: {node} | Pressão: {press:.2f} m')


Nó: Leak_P17 | Pressão: 1136.23 m
Nó: N326 | Pressão: 48.29 m
Nó: N18 | Pressão: 46.49 m
Nó: N191 | Pressão: 46.47 m
Nó: N192 | Pressão: 46.43 m
Nó: N187 | Pressão: 46.40 m
Nó: N19 | Pressão: 46.38 m
Nó: N188 | Pressão: 46.10 m
Nó: N64 | Pressão: 45.86 m
Nó: N189 | Pressão: 45.84 m
Nó: N186 | Pressão: 45.78 m
Nó: N65 | Pressão: 45.63 m
Nó: N190 | Pressão: 45.51 m
Nó: N193 | Pressão: 45.34 m
Nó: N185 | Pressão: 45.23 m
Nó: N353 | Pressão: 44.45 m
Nó: N207 | Pressão: 44.30 m
Nó: N20 | Pressão: 44.17 m
Nó: N194 | Pressão: 44.06 m
Nó: N352 | Pressão: 43.84 m
Nó: N17 | Pressão: 43.38 m
Nó: N195 | Pressão: 43.20 m
Nó: N203 | Pressão: 43.17 m
Nó: N325 | Pressão: 42.93 m
Nó: N350 | Pressão: 42.88 m
Nó: N202 | Pressão: 42.74 m
Nó: N206 | Pressão: 42.65 m
Nó: N21 | Pressão: 42.64 m
Nó: N176 | Pressão: 42.49 m
Nó: N196 | Pressão: 42.41 m
Nó: N177 | Pressão: 42.28 m
Nó: N175 | Pressão: 42.22 m
Nó: N174 | Pressão: 42.17 m
Nó: N178 | Pressão: 42.16 m
Nó: N327 | Pressão: 42.12 m
Nó: N63 | Pressão: 42

In [50]:
import numpy as np
from scipy.stats import lognorm
import networkx as nx
import geopandas as gpd
import matplotlib.pylab as plt
import warnings
import wntr
import folium
from pyproj import Transformer
# Create a WaterNetworkModel from an EPANET INP file

wn = wntr.network.WaterNetworkModel('Sim_PKW_Q28.inp')
nome_reservatorio = 'R1'         # Nome do reservatório
base_head = 1153.95                # Carga hidráulica (ajuste conforme sua rede)
coordinates = (-47.971286,-15.927268)             # Coordenadas (opcional, ajuste conforme necessário)
# Parâmetros da tubulação
nome_tubulacao = 'P_R1_N35'
comprimento = 0.10                # Comprimento em metros (ajuste conforme necessário)
diametro = 110                   # Diâmetro em mm (ajuste conforme necessário)
rugosidade = 140                 # Rugosidade (ajuste conforme necessário)

# Adiciona o reservatório à rede
wn.add_reservoir(nome_reservatorio, base_head=base_head, coordinates=coordinates)
# Adiciona a tubulação conectando o reservatório ao nó N35
wn.add_pipe(nome_tubulacao, nome_reservatorio, 'N35', length=comprimento, diameter=diametro, roughness=rugosidade)


---------------------------------------

In [31]:
import folium
# Crie o mapa centralizado nas coordenadas do reservatório
m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=16)

# Adicione o reservatório com um marcador especial (cor diferente)
folium.Marker(
    location=[coordinates[1], coordinates[0]],
    popup=nome_reservatorio,
    icon=folium.CustomIcon('https://cdn-icons-png.flaticon.com/512/5733/5733770.png',
    icon_size=(20, 20),
)
).add_to(m)

# Adicione os nós restantes da rede (exemplo para todos os nós)
for node_name, node in wn.nodes():
    if node_name != nome_reservatorio:
        if hasattr(node, 'coordinates') and node.coordinates:
            folium.CircleMarker(
                location=[node.coordinates[1], node.coordinates[0]],
                radius=4,
                color='red',
                fill=True,
                fill_color='blue',
                popup=node_name
            ).add_to(m)

# Adicione as tubulações (pipes) como linhas
for pipe_name, pipe in wn.pipes():
    start_node = wn.get_node(pipe.start_node_name)
    end_node = wn.get_node(pipe.end_node_name)
    if hasattr(start_node, 'coordinates') and hasattr(end_node, 'coordinates'):
        folium.PolyLine(
            locations=[
                [start_node.coordinates[1], start_node.coordinates[0]],
                [end_node.coordinates[1], end_node.coordinates[0]]
            ],
            color='gray',
            weight=2,
            popup=pipe_name
        ).add_to(m)

# Salve o mapa em HTML
m.save('rede_mapa_personalizado.html')
